In [19]:
import os
import numpy as np
from PIL import Image
import torch
from torchvision import transforms, models
import torch.nn.functional as F
import json
import requests
import pandas as pd

import pickle
from tqdm import tqdm

In [20]:
# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the transformation for the images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [21]:
# Download the ImageNet class index to label mapping
class_idx = json.loads(requests.get('https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json').text)
idx_to_class = {int(k): v[1] for k, v in class_idx.items()}

In [22]:
# Load a pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
model = model.to(device)
model.eval()

C:\Users\richt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\richt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [23]:
""" def load_images(base_path, n_assemblies=None):
    all_images = {}
    assembly_folders = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    
    # Limit the number of assemblies if specified
    if n_assemblies is not None:
        assembly_folders = assembly_folders[:n_assemblies]

    for assembly_id in assembly_folders:
        assembly_path = os.path.join(base_path, assembly_id)
        all_images[assembly_id] = {
            'assembly': None,
            'bodies': {}
        }
    
        for image_name in os.listdir(assembly_path):
            if not image_name.endswith('.png'):
                continue
            image_path = os.path.join(assembly_path, image_name)
            with Image.open(image_path) as img:
                if image_name == 'assembly.png':
                    all_images[assembly_id]['assembly'] = np.array(img)
                else:
                    body_id = image_name.rstrip('.png')
                    all_images[assembly_id]['bodies'][body_id] = np.array(img)

    return all_images
 """

def load_images(base_path, resize_shape=(256, 256), n_assemblies=None):
    all_images = {}
    assembly_folders = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    
    # Limit the number of assemblies if specified
    if n_assemblies is not None:
        assembly_folders = assembly_folders[:n_assemblies]

    for assembly_id in tqdm(assembly_folders, desc="Loading images"):
        assembly_path = os.path.join(base_path, assembly_id)
        all_images[assembly_id] = {
            'assembly': None,
            'bodies': {}
        }
    
        for image_name in os.listdir(assembly_path):
            if not image_name.endswith('.png'):
                continue
            image_path = os.path.join(assembly_path, image_name)
            with Image.open(image_path) as img:
                img = img.resize(resize_shape)
                if image_name == 'assembly.png':
                    all_images[assembly_id]['assembly'] = np.array(img)
                else:
                    body_id = image_name.rstrip('.png')
                    all_images[assembly_id]['bodies'][body_id] = np.array(img)

    return all_images


In [24]:
def predict_image(image, model, transform, topk=5):
    # If image is a path, then load it; otherwise, assume it's already an array
    if isinstance(image, str):
        image = Image.open(image)
    else:
        image = Image.fromarray(image)
    
    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)
    
    with torch.no_grad():
        output = model(image)
        probs, indices = output.topk(topk)
        probs = F.softmax(probs, dim=1)[0] * 100
        indices = indices[0]
    
    labels = [idx_to_class[int(idx.item())] for idx in indices]
    return labels, probs.cpu().numpy()

In [25]:
def predict_for_images(all_images, model, transform):
    predictions = {}
    
    for assembly_id, images in all_images.items():
        predictions[assembly_id] = {
            'assembly': None,
            'bodies': {}
        }
        
        # Predict for assembly image
        labels, probabilities = predict_image(images['assembly'], model, transform)
        predictions[assembly_id]['assembly'] = (labels, probabilities)
        
        # Predict for body images
        for body_id, body_image in images['bodies'].items():
            labels, probabilities = predict_image(body_image, model, transform)
            predictions[assembly_id]['bodies'][body_id] = (labels, probabilities)
            
    return predictions

In [26]:
def predictions_to_dataframe(all_predictions, topk=5):
    # Lists to store data for assemblies and bodies
    assembly_data = []
    body_data = []

    for assembly_id, predictions in all_predictions.items():
        # Process assembly predictions
        assembly_labels, assembly_probs = predictions['assembly']
        assembly_dict = {
            'assembly_id': assembly_id,
        }
        for i in range(topk):
            assembly_dict[f'assembly_prediction_{i + 1}'] = assembly_labels[i]
            assembly_dict[f'assembly_probability_{i + 1}'] = assembly_probs[i]
        assembly_data.append(assembly_dict)

        # Process body predictions
        for body_id, (body_labels, body_probs) in predictions['bodies'].items():
            body_dict = {
                'assembly_id': assembly_id,
                'body_id': body_id
            }
            for i in range(topk):
                body_dict[f'prediction_{i + 1}'] = body_labels[i]
                body_dict[f'probability_{i + 1}'] = body_probs[i]
            body_data.append(body_dict)

    # Convert lists to dataframes
    df_assembly_predictions = pd.DataFrame(assembly_data)
    df_body_predictions = pd.DataFrame(body_data)
    
    return df_assembly_predictions, df_body_predictions

In [27]:
""" # Load images
base_path = "C:\\Users\\richt\\Documents\\ASME_data\\train\\Fusion360GalleryDataset_23hackathon_train"
all_images = load_images(base_path, resize_shape=(128, 128)) """

' # Load images\nbase_path = "C:\\Users\\richt\\Documents\\ASME_data\\train\\Fusion360GalleryDataset_23hackathon_train"\nall_images = load_images(base_path, resize_shape=(128, 128)) '

In [28]:
# Load images
base_path_test = "C:\\Users\\richt\\Documents\\ASME_data\\test\\Fusion360GalleryDataset_23hackathon_test"
all_images_test = load_images(base_path_test)

Loading images: 100%|██████████| 10/10 [00:03<00:00,  2.97it/s]


In [29]:
""" # To save the all_images dictionary to a file:
with open("all_images.pkl", "wb") as f:
    pickle.dump(all_images, f) """

' # To save the all_images dictionary to a file:\nwith open("all_images.pkl", "wb") as f:\n    pickle.dump(all_images, f) '

In [30]:
# To save the all_images dictionary to a file:
with open("all_images_test.pkl", "wb") as f:
    pickle.dump(all_images_test, f)

In [31]:
""" import matplotlib.pyplot as plt

def visualize_assemblies_and_bodies(images_dict):
    # Selecting the first 3 assemblies
    assemblies = list(images_dict.keys())[:3]
    
    fig, axes = plt.subplots(3, 5, figsize=(20, 15))  # 3 assemblies, up to 5 images each (assembly + up to 4 bodies)
    
    for i, assembly in enumerate(assemblies):
        # Displaying assembly image
        axes[i, 0].imshow(images_dict[assembly]['assembly'], cmap='gray')
        axes[i, 0].set_title(f"{assembly} - Assembly")
        axes[i, 0].axis('off')

        # Displaying up to the first 4 body images if they exist
        body_ids = list(images_dict[assembly]['bodies'].keys())
        for j in range(4):
            if j < len(body_ids):  # if there's a body to display
                axes[i, j+1].imshow(images_dict[assembly]['bodies'][body_ids[j]], cmap='gray')
                axes[i, j+1].set_title(f"{assembly} - Body {body_ids[j]}")
                axes[i, j+1].axis('off')
            else:
                axes[i, j+1].axis('off')  # Turn off axis if no more bodies to display

    plt.tight_layout()
    plt.show()

# Now, let's call this function to visualize the images
visualize_assemblies_and_bodies(all_images)
 """

' import matplotlib.pyplot as plt\n\ndef visualize_assemblies_and_bodies(images_dict):\n    # Selecting the first 3 assemblies\n    assemblies = list(images_dict.keys())[:3]\n    \n    fig, axes = plt.subplots(3, 5, figsize=(20, 15))  # 3 assemblies, up to 5 images each (assembly + up to 4 bodies)\n    \n    for i, assembly in enumerate(assemblies):\n        # Displaying assembly image\n        axes[i, 0].imshow(images_dict[assembly][\'assembly\'], cmap=\'gray\')\n        axes[i, 0].set_title(f"{assembly} - Assembly")\n        axes[i, 0].axis(\'off\')\n\n        # Displaying up to the first 4 body images if they exist\n        body_ids = list(images_dict[assembly][\'bodies\'].keys())\n        for j in range(4):\n            if j < len(body_ids):  # if there\'s a body to display\n                axes[i, j+1].imshow(images_dict[assembly][\'bodies\'][body_ids[j]], cmap=\'gray\')\n                axes[i, j+1].set_title(f"{assembly} - Body {body_ids[j]}")\n                axes[i, j+1].axis(

In [32]:
""" # Make predictions
all_predictions = predict_for_images(all_images, model, transform)

# [Further code for storing predictions and merging with df_main if needed] """

' # Make predictions\nall_predictions = predict_for_images(all_images, model, transform)\n\n# [Further code for storing predictions and merging with df_main if needed] '

In [33]:
all_test_predictions = predict_for_images(all_images_test, model, transform)

In [34]:
""" # Convert predictions to dataframes
df_assembly_predictions, df_body_predictions = predictions_to_dataframe(all_predictions) """

' # Convert predictions to dataframes\ndf_assembly_predictions, df_body_predictions = predictions_to_dataframe(all_predictions) '

In [35]:
# Convert predictions to dataframes
df_assembly_predictions_test, df_body_predictions_test = predictions_to_dataframe(all_test_predictions)

In [36]:
df_assembly_predictions_test.tail(5), df_body_predictions_test.tail()

(      assembly_id assembly_prediction_1  assembly_probability_1  \
 5  35584_fb213b6b               padlock               46.227245   
 6  43630_515b0cd2         rocking_chair               41.330933   
 7  55199_4d57661f                 loupe               35.880558   
 8  72966_11b78e5e      pencil_sharpener               52.291607   
 9  74576_73ac0093          dining_table               94.939171   
 
   assembly_prediction_2  assembly_probability_2 assembly_prediction_3  \
 5      combination_lock               43.865902               lighter   
 6          barber_chair               18.414169              bassinet   
 7               barbell               33.910595      magnetic_compass   
 8           face_powder               21.584303              dumbbell   
 9         folding_chair                2.509799                  desk   
 
    assembly_probability_3 assembly_prediction_4  assembly_probability_4  \
 5                5.058557                 loupe                3.76

In [37]:
""" # Loading the df_main.pickle file
df_main = pd.read_pickle("df_main.pkl")
df_main.shape """

' # Loading the df_main.pickle file\ndf_main = pd.read_pickle("df_main.pkl")\ndf_main.shape '

In [38]:
# Loading the df_main.pickle file
df_main_test = pd.read_pickle("df_test_main_fe.pkl")
df_main_test.shape

(155, 38)

In [39]:
""" # Start with df_main
merged_df = df_main.copy()

# Merge with assembly predictions
merged_df = merged_df.merge(df_assembly_predictions, on='assembly_id', how='left')

# Merge with body predictions
merged_df = merged_df.merge(df_body_predictions, on=['assembly_id', 'body_id'], how='left', suffixes=('', '_body'))

# Handle any NaN values that might arise due to missing predictions
for col in merged_df.columns:
    if 'prediction' in col or 'probability' in col:
        merged_df[col].fillna('', inplace=True) """

" # Start with df_main\nmerged_df = df_main.copy()\n\n# Merge with assembly predictions\nmerged_df = merged_df.merge(df_assembly_predictions, on='assembly_id', how='left')\n\n# Merge with body predictions\nmerged_df = merged_df.merge(df_body_predictions, on=['assembly_id', 'body_id'], how='left', suffixes=('', '_body'))\n\n# Handle any NaN values that might arise due to missing predictions\nfor col in merged_df.columns:\n    if 'prediction' in col or 'probability' in col:\n        merged_df[col].fillna('', inplace=True) "

In [40]:
# Start with df_main
merged_df_test = df_main_test.copy()

# Merge with assembly predictions
merged_df_test = merged_df_test.merge(df_assembly_predictions_test, on='assembly_id', how='left')

# Merge with body predictions
merged_df_test = merged_df_test.merge(df_body_predictions_test, on=['assembly_id', 'body_id'], how='left', suffixes=('', '_body'))

# Handle any NaN values that might arise due to missing predictions
for col in merged_df_test.columns:
    if 'prediction' in col or 'probability' in col:
        merged_df_test[col].fillna('', inplace=True)

In [41]:
""" pd.set_option('display.max_columns', None)
merged_df.tail(10) """

" pd.set_option('display.max_columns', None)\nmerged_df.tail(10) "

In [42]:
pd.set_option('display.max_columns', None)
merged_df_test.tail(10)#
merged_df_test.shape

(155, 58)

In [43]:
""" merged_df.to_pickle("df_main_images.pkl") """

' merged_df.to_pickle("df_main_images.pkl") '

In [44]:
merged_df_test.to_pickle("df_main_images_test.pkl")